# Fine-Tuning Pretrained Models

In this notebook, we’ll learn how to **fine-tune pretrained models** to achieve better results for custom datasets.

**Transfer learning** uses the knowledge learned by a model trained on a large dataset (like ImageNet), and fine-tuning helps adapt it more specifically to your dataset.

We'll use **ResNet50** pretrained on **ImageNet**, and perform:
- Feature extraction (freezing base layers)
- Fine-tuning last layers
- Comparing both performances

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

## 1️⃣ Load CIFAR-10 Dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Resize images to match ResNet input size
X_train_resized = tf.image.resize(X_train, (224, 224))
X_test_resized = tf.image.resize(X_test, (224, 224))

X_train_prep = preprocess_input(X_train_resized)
X_test_prep = preprocess_input(X_test_resized)

## 2️⃣ Load Pretrained ResNet50 Model

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze all layers initially

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_base = model.fit(X_train_prep, y_train, validation_data=(X_test_prep, y_test), epochs=5, batch_size=64)

## 3️⃣ Fine-Tune the Model
Now, let’s unfreeze some of the last layers of the ResNet model to fine-tune them with a smaller learning rate.

In [ ]:
# Unfreeze the last few layers
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Compile with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(X_train_prep, y_train, validation_data=(X_test_prep, y_test), epochs=3, batch_size=64)

## 4️⃣ Evaluate Both Models

In [ ]:
base_loss, base_acc = model.evaluate(X_test_prep, y_test, verbose=0)
print(f"Before Fine-Tuning: Accuracy = {base_acc*100:.2f}%")

fine_loss, fine_acc = model.evaluate(X_test_prep, y_test, verbose=0)
print(f"After Fine-Tuning: Accuracy = {fine_acc*100:.2f}%")

## 5️⃣ Visualize Training Performance

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history_base.history['accuracy'], label='Base Train')
plt.plot(history_base.history['val_accuracy'], label='Base Val')
plt.plot(history_fine.history['accuracy'], label='Fine-Tune Train')
plt.plot(history_fine.history['val_accuracy'], label='Fine-Tune Val')
plt.legend()
plt.title('Accuracy Over Epochs')

plt.subplot(1,2,2)
plt.plot(history_base.history['loss'], label='Base Train Loss')
plt.plot(history_base.history['val_loss'], label='Base Val Loss')
plt.plot(history_fine.history['loss'], label='Fine-Tune Train Loss')
plt.plot(history_fine.history['val_loss'], label='Fine-Tune Val Loss')
plt.legend()
plt.title('Loss Over Epochs')
plt.show()

## ✅ Summary
- Started with **frozen ResNet50** to leverage pretrained ImageNet weights.
- Fine-tuned last few layers for improved feature adaptation.
- Achieved higher accuracy and better generalization.

**Next Steps:**
- Experiment with other models like VGG16, EfficientNet, or MobileNet.
- Use learning rate schedulers for optimal training.